# *FEniCS NB:* Laser Pulse Crossing a Crystal
### *File:*    fn_simp_06a.py — propagate only the laser pulse

The Frantz-Nodvik equations
$$
\begin{align}
  \partial_t n + c\partial x &= \sigma c n \Delta, \\
  \partial_t \Delta          &= -\gamma \sigma c n \Delta, \\
\end{align}
$$
are a pair of coupled, first-order PDEs that describe
the propagation of a one-dimensional laser pulse across a
slab of laser gain material composed of two-level atoms.
Here, $x$ and $t$ denote respectively distance along the beam axis and
the time; $n(x,t)$ denotes photon number density in the medium;
and $\Delta(x,t)$ denotes the “population inversion”, $N_2 - N_1$,
giving the difference between the number density of atoms
in the excited state as compared to those in the ground state.
In addition, $c$ denotes the speed of light in the medium,
$\sigma$ the resonance absorption cross-section,
and $\gamma$ a factor related to the relative degeneracy
of the ground and excited states.

One set of boundary conditions relevant to this problem are
the initial population inversion over the domain of interest,
together with the initial profile of the incident laser pulse.

The only dynamical equation is that of the excited states in the crystal.

Experimenting with different solver here...

File:  fn_simp_01.py

This simple test shows an initial laser pulse envelope.
The envelope propagates to the right, while also dispersing.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# import os
# import sys
# from IPython.display import Image
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from fenics import *
from mshr import *

Describe the simulation: parameters, domain, boundary conditions, initial conditions.

In [ ]:
# set simulation parameters: domain, crystal, laser, time step
# -- simulation domain
Ld = 1.00         # length of simulation domain (we force, see below, domain to [0,1])
mesh_d = 226      # mesh density
ds = Ld / mesh_d  # size of individual mesh cells
# -- crystal
Lc = 100 * ds     # length of crystal
D0 = 10.0         # initial population inversion density
# -- laser pulse (normalized parameters)
cl = 1.00         # speed of light in vacuum
cm = 163 * ds     # speed of light in crystal
lp_w = 63 * ds    # laser pulse width
lp_hw = lp_w / 2  # laser pulse half-width
lp_x0 = 0.10      # laser pulse starting point
lp_n0 = 1.00      # incident photon density
# -- time step
T = 183 * ds      # total simulation time
n_steps = 201     # number of time steps
dt = T / n_steps  # size of time step
nip = 50          # number of intervals between plots

# create domain: a 1D mesh on the unit interval [0,1]
mesh = UnitIntervalMesh(mesh_d)
V = FunctionSpace(mesh, "CG", 1)

# define boundary functions: left and right ends
def on_L(x, on_boundary):
    return (on_boundary and near(x[0], 0.))

def on_R(x, on_boundary):
    return (on_boundary and near (x[0], 1.))
    
# specify boundary conditions
bc_L = DirichletBC(V, Constant(0), on_L)
bc_R = DirichletBC(V, Constant(0), on_R)
bc = [bc_L, bc_R]
# bc = bc_L

# specify initial conditions
# -- define profile of incident laser pulse: half wave near left edge of domain
nph_init = Expression(('x[0]>2.*ds && x[0]<(lp_w-2.*ds) ? \
                        lp_n0*sin(pi*(x[0]-2.*ds)/(lp_w-4.*ds)) : 0.'), \
                       degree=1, lp_w=0.2, Lc=Lc, lp_n0=lp_n0, ds=ds)
# -- initialize nph_j = nph_init
nph_j = interpolate(nph_init, V)

Set up the variational problem.

In [ ]:
# define variational problem
nph = TrialFunction(V)
v   = TestFunction(V)
F = ( nph - nph_j + cm * dt * nph.dx(0) ) * v * dx
a, L = lhs(F), rhs(F)

Execute simulation.

In [ ]:
# initialization
nph = Function(V)
t = 0

# time steps
for ii in range(n_steps):

    # update time
    t += dt

    # compute solution
    solve(a == L, nph, bc)

    # update plot at nip-step intervals
    if ii % nip == 0:
        plot(nph)

    # update previous solution
    nph_j.assign(nph)

# final envelope of laser pulse (if necessary)
if ii % nip != 0:
    plot(nph)

# finish plot
# plt.xlim([0.00, 0.20])
# plt.ylim([0.00, 0.20])
plt.grid(True)
plt.show()
plt.close()

The following approach is useful for 2D and 3D systems,
when overlaying plots may not make sense.

In [ ]:
# define time-evolution function
def evolve():

    # initialization
    nph = Function(V)
    t = 0

    # time steps
    for ii in range(n_steps):

        # update current time
        t += dt

        # compute solution
        solve(a == L, nph, bc)

        # report solution at nip-step intervals
        if ii % nip == 0:
            yield nph

        # update previous solution
        nph_j.assign(nph)

    # final envelope of laser pulse (if necessary)
    if ii % nip == 0:
        yield nph

In [ ]:
n_rows = 3
n_cols = 5
fig_wd = 15
# default sizing here yields unit aspect ratio
plt.figure(figsize = (fig_wd, fig_wd * n_rows // n_cols))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

idx = 0
for u in evolve():
    idx += 1
    plt.subplot(n_rows, n_cols, idx)
    plot(u)
#     plot(u, vmin=u_min, vmax=u_max)